### OpenAI predictions and performance for updated training set ###
We have updates on data sets and prompts.
Use this code as a template to run predictions for the trainin and test sets

In [1]:
import os
import copy
import numpy as np
import pandas as pd
import time
from collections import deque

# Appearance of the Notebook
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
np.set_printoptions(linewidth=110)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)

# Import this module with autoreload
%load_ext autoreload
%autoreload 2
from llmt.openaimodel import OpenAIModel
from llmt.performance import Performance

In [2]:
data_dir = os.path.join(os.environ.get('DATA'), 'hcp')
print(data_dir)
train_file_name = 'hcp-train-250701.parquet'
train_file = os.path.join(data_dir, train_file_name)
print(train_file)
train_df = pd.read_parquet(train_file)
print(train_df.shape)
print(len(train_df['id'].unique()))
display(train_df.head(2))
company_id_list = list(train_df['id'].unique())
print(len(company_id_list))

/app/data/hcp
/app/data/hcp/hcp-train-250701.parquet
(187, 7)
187


,id,name,description,mental_health,inpatient,outpatient,dset
0,491639-77,144-Bed Hospital Facility Joint Venture in Den...,Provider of behavioral health services located...,1,1,1,train
1,10995-58,Acadia Healthcare (NAS: ACHC),Acadia Healthcare Co Inc acquires and develops...,1,1,1,train


187


In [3]:
# Define the names of the true and predicted columns
# We need to define these names somewhere to combine the correct columns
true_col_dict = {'true_mh_col': 'mental_health',
                 'true_ip_col': 'inpatient',
                 'true_op_col': 'outpatient',
                 'true_mh_ip_col': 'mh_ip',
                 'true_mh_op_col': 'mh_op'}

pred_col_dict = {'pred_mh_col': 'pred_mh',
                 'pred_ip_col': 'pred_ip',
                 'pred_op_col': 'pred_op',
                 'pred_mh_ip_col': 'pred_mh_ip',
                 'pred_mh_op_col': 'pred_mh_op'}

col_dict = copy.deepcopy(true_col_dict)
col_dict.update(pred_col_dict)

# Create a new dictionary with the values above
# We need this dictionary to run a loop over the true/pred pairs
true_keys = true_col_dict.keys()
pred_keys = [key.replace('true', 'pred') for key in true_keys]
true_col_list = [true_col_dict.get(k) for k in true_keys]
pred_col_list = [pred_col_dict.get(k) for k in pred_keys]
true_pred_cols = dict(zip(true_col_list, pred_col_list))
display(true_pred_cols)

{'mental_health': 'pred_mh',
 'inpatient': 'pred_ip',
 'outpatient': 'pred_op',
 'mh_ip': 'pred_mh_ip',
 'mh_op': 'pred_mh_op'}

In [36]:
print(true_col_list)
print(pred_col_list)

['mental_health', 'inpatient', 'outpatient', 'mh_ip', 'mh_op']
['pred_mh', 'pred_ip', 'pred_op', 'pred_mh_ip', 'pred_mh_op']


In [18]:
# API endpoint
print(os.environ.get('OPENAI_API_ENDPOINT'))
print(os.environ.get('OPENAI_API_VERSION'))

https://azure-ai.hms.edu
2025-04-01-preview


### Functions ###

In [4]:
def rotate_columns(data, by=1):
    col_deque = deque(list(data.columns))
    col_deque.rotate(by)
    return data[list(col_deque)]

# Run the prediction functions for a specific model
def predict_classes(model, name, description, temperature, version_dict=None):
    if version_dict is None:
        version_dict = {'mh': 3, 'ip': 1, 'op': 1}
    response_list = []
    response_mh = model.predict_mh(name=name, 
                                   description=description, 
                                   temperature=temperature, 
                                   version=version_dict.get('mh'))
    response_list.append(response_mh)
    response_ip = model.predict_ip(name=name, 
                                   description=description, 
                                   temperature=temperature, 
                                   version=version_dict.get('ip'))
    response_list.append(response_ip)
    response_op = model.predict_op(name=name, 
                                   description=description, 
                                   temperature=temperature, 
                                   version=version_dict.get('op'))
    response_list.append(response_op)
    assert len(response_list) == 3
    return response_list

# Combine mental health with inpatient/outpatient columns
def combine_mental_health_columns(data, col_dict):
    ''' Combine mental health with inpatient/outpatient services columns '''
    data_ip = Performance(data=data).\
        combine_columns(true_col_list=[col_dict.get('true_mh_col'), col_dict.get('true_ip_col')], 
                        pred_col_list=[col_dict.get('pred_mh_col'), col_dict.get('pred_ip_col')])
    
    data_op = Performance(data=data_ip).\
        combine_columns(true_col_list=[col_dict.get('true_mh_col'), col_dict.get('true_op_col')], 
                        pred_col_list=[col_dict.get('pred_mh_col'), col_dict.get('pred_op_col')])
    output = data_op.\
        rename(columns={f'{col_dict.get('true_mh_col')}_{col_dict.get('true_ip_col')}': col_dict.get('true_mh_ip_col'),
                        f'{col_dict.get('true_mh_col')}_{col_dict.get('true_op_col')}': col_dict.get('true_mh_op_col'),
                        f'{col_dict.get('pred_mh_col')}_{col_dict.get('pred_ip_col')}': col_dict.get('pred_mh_ip_col'),
                        f'{col_dict.get('pred_mh_col')}_{col_dict.get('pred_op_col')}': col_dict.get('pred_mh_op_col')})
    return output

def performance_table(data, true_pred_cols):
    performance_df_list = []
    for v, (true_col, pred_col) in enumerate(true_pred_cols.items()):
        performance_dict = Performance(data=results_df).binary_performance(true_col=true_col, pred_col=pred_col)
        performance_df = pd.DataFrame(performance_dict, index=[v]).\
            assign(category=true_col)
        performance_df_list.append(rotate_columns(performance_df))
    performance_df = pd.concat(performance_df_list, axis=0)
    return performance_df

### Code to prepare model prompts and outputs ###

In [73]:
df = copy.deepcopy(train_df)
display(df.head(2))

# Run an example ID, just to make sure it's all working
idx = 26
company_id = df['id'].unique()[idx]
company_df = df.loc[df['id'] == company_id]
name = company_df.get('name').values[0]
description = company_df.get('description').values[0]
print()
print(name)
print()
print(description)

version_dict_1 = {'mh': 3, 'ip': 1, 'op': 1}
version_dict_2 = {'mh': 4, 'ip': 2, 'op': 2}

deployment_name_1 = 'gpt-4o-1120'
deployment_name_2 = 'gpt-4.1'

model = OpenAIModel(model=deployment_name_1)
temperature = 0
response_list = predict_classes(model=model, 
                                name=name, 
                                description=description, 
                                temperature=temperature, 
                                version_dict=version_dict_1)

# Add the responses to the company_df dataframe
for response in response_list:
    company_df = company_df.assign(**response)

# Create the combination of columns
company_df = Performance(data=company_df).\
    combine_columns(true_col_list=['mental_health', 'inpatient'],
                    pred_col_list=['pred_mh', 'pred_ip']).\
    rename(columns={'pred_mh_pred_ip': 'pred_mh_ip'})

company_df = Performance(data=company_df).\
    combine_columns(true_col_list=['mental_health', 'outpatient'],
                    pred_col_list=['pred_mh', 'pred_op']).\
    rename(columns={'pred_mh_pred_op': 'pred_mh_op'})

print()
print(response_list)
print()
display(company_df)

,id,name,description,mental_health,inpatient,outpatient,dset
0,491639-77,144-Bed Hospital Facility Joint Venture in Den...,Provider of behavioral health services located...,1,1,1,train
1,10995-58,Acadia Healthcare (NAS: ACHC),Acadia Healthcare Co Inc acquires and develops...,1,1,1,train



Bayonne Medical Center

Provider of acute-care services based in Bayonne, New Jersey. The company operates as a hospital and offers behavioral health care, cardiac care, emergency room, laboratory, outpatient and inpatient services.

[{'pred_mh': 1}, {'pred_ip': 1}, {'pred_op': 1, 'verified_op': 1}]



,id,name,description,mental_health,inpatient,outpatient,dset,pred_mh,pred_ip,pred_op,verified_op,mental_health_inpatient,pred_mh_ip,mental_health_outpatient,pred_mh_op
0,125146-99,Bayonne Medical Center,Provider of acute-care services based in Bayon...,1,1,1,train,1,1,1,1,1,1,1,1


In [74]:
# Function to send a sample to the model
def send_sample(company_df, deployment_name, version_dict):
    assert len(company_df) == 1, 'The input data frame is one sample and should have just ONE ROW!'
    company_name = company_df.get('name').values[0]
    company_description = company_df.get('description').values[0]
    model = OpenAIModel(model=deployment_name)
    response_list = predict_classes(model=model, 
                                    name=company_name, 
                                    description=company_description, 
                                    temperature=0, 
                                    version_dict=version_dict)
    # Add the responses to the company_df dataframe
    for response in response_list:
        company_df = company_df.assign(**response)
    # Combine the binary columns
    company_df = Performance(data=company_df).\
        combine_columns(true_col_list=['mental_health', 'inpatient'],
                        pred_col_list=['pred_mh', 'pred_ip']).\
        rename(columns={'pred_mh_pred_ip': 'pred_mh_ip'})

    company_df = Performance(data=company_df).\
        combine_columns(true_col_list=['mental_health', 'outpatient'],
                        pred_col_list=['pred_mh', 'pred_op']).\
        rename(columns={'pred_mh_pred_op': 'pred_mh_op'})
    
    return company_df

# Try this function
company_df = df.loc[df['id'] == company_id]
test_df = send_sample(company_df=company_df, deployment_name=deployment_name_1, version_dict=version_dict_1)
display(test_df)

,id,name,description,mental_health,inpatient,outpatient,dset,pred_mh,pred_ip,pred_op,verified_op,mental_health_inpatient,pred_mh_ip,mental_health_outpatient,pred_mh_op
0,125146-99,Bayonne Medical Center,Provider of acute-care services based in Bayon...,1,1,1,train,1,1,1,1,1,1,1,1


### Run models on Training data with evaluations ###

In [86]:
# Information required to run the loops
df = copy.deepcopy(train_df)
company_id_list = sorted(list(df['id'].unique()))
deployment_name_list = ['gpt-4o-1120', 'gpt-4.1']
version_dict_list = [{'mh': 3, 'ip': 1, 'op': 1}, {'mh': 4, 'ip': 2, 'op': 2}]

# LOOP OVER MODELS
results_samples_df_list = []
for d, deployment_name in enumerate(deployment_name_list):
    print()
    print(f'Running model  {d+1}/{len(deployment_name_list)}: {deployment_name}')
    
    # LOOP OVER PROMPT VERSIONS
    for p, prompt_version_dict in enumerate(version_dict_list):   
        print()
        print(f'Prompt version {p+1}/{len(version_dict_list)}: {prompt_version_dict}')

        # LOOP OVER THE SAMPLES
        results_samples_list = []
        for c, company_id in enumerate(company_id_list[:6]):
            if (c+1) % 2 == 0:
                print(f'Sample {c+1}/{len(company_id_list)}')
            company_df = df.loc[df['id'] == company_id]
            company_name = company_df.get('name').values[0]
            company_desc = company_df.get('description').values[0]
            result_df = send_sample(company_df=company_df, 
                                    deployment_name=deployment_name, 
                                    version_dict=prompt_version_dict)
            results_samples_list.append(result_df)
        # Assemble the results for this model/prompt combination
        results_samples = pd.concat(results_samples_list, axis=0, ignore_index=True)
        results_samples.insert(loc=0, column='model', value=deployment_name)
        results_samples.insert(loc=1, column='prompt', value=p+1)
        results_samples_df_list.append(results_samples)
        # ToDo: Model performance for this combination 
results_samples_df = pd.concat(results_samples_df_list, axis=0, ignore_index=True)


Running model  1/2: gpt-4o-1120

Prompt version 1/2: {'mh': 3, 'ip': 1, 'op': 1}
Sample 2/187
Sample 4/187
Sample 6/187

Prompt version 2/2: {'mh': 4, 'ip': 2, 'op': 2}
Sample 2/187
Sample 4/187
Sample 6/187

Running model  2/2: gpt-4.1

Prompt version 1/2: {'mh': 3, 'ip': 1, 'op': 1}
Sample 2/187
Sample 4/187
Sample 6/187

Prompt version 2/2: {'mh': 4, 'ip': 2, 'op': 2}
Sample 2/187
Sample 4/187
Sample 6/187


In [87]:
results_samples_df

,model,prompt,id,name,description,mental_health,inpatient,outpatient,dset,pred_mh,pred_ip,pred_op,verified_op,mental_health_inpatient,pred_mh_ip,mental_health_outpatient,pred_mh_op
0,gpt-4o-1120,1,100363-69,Chilton Hospital,Operator of inpatient hospital center. The com...,1,1,1,train,1,1,0,1,1,1,1,0
1,gpt-4o-1120,1,10127-62,Quest Diagnostics (NYS: DGX),Quest Diagnostics is a leading independent pro...,1,0,0,train,0,0,0,1,0,0,0,0
2,gpt-4o-1120,1,10195-21,IASIS Healthcare,Owner and operator of medium-sized acute care ...,1,1,1,train,1,1,1,1,1,1,1,1
3,gpt-4o-1120,1,10468-27,Behavioral Centers of America,Provider of psychiatric healthcare services. T...,1,1,1,train,1,1,1,1,1,1,1,1
4,gpt-4o-1120,1,107240-50,Alvarado Parkway Institute,Operator of a psychiatric health care facility...,1,1,1,train,1,1,1,1,1,1,1,1
5,gpt-4o-1120,1,107624-62,Baldwin County Mental Health Center,Provider of mental health services based out o...,1,0,1,train,1,0,1,1,0,0,1,1
6,gpt-4o-1120,2,100363-69,Chilton Hospital,Operator of inpatient hospital center. The com...,1,1,1,train,1,1,0,1,1,1,1,0
7,gpt-4o-1120,2,10127-62,Quest Diagnostics (NYS: DGX),Quest Diagnostics is a leading independent pro...,1,0,0,train,0,0,0,1,0,0,0,0
8,gpt-4o-1120,2,10195-21,IASIS Healthcare,Owner and operator of medium-sized acute care ...,1,1,1,train,0,1,1,1,1,0,1,0
9,gpt-4o-1120,2,10468-27,Behavioral Centers of America,Provider of psychiatric healthcare services. T...,1,1,1,train,1,1,1,1,1,1,1,1
